In [ ]:
#Nombre: Mikel Alvarez Bejarano
#Carrera: Ing. Sistemas
#Materia: Sis-420
#Examen Segunda instancia
#Fecha: 07/07/2023

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sensioai/blog/blob/master/025_mlp_framework/mlp_framework.ipynb)

In [1]:
class MLP:
    #método que inicializa una instancia de la clase
    def __init__(self, layers):
        # el MLP es una lista de capas
        self.layers = layers

    def __call__(self, x):
        # calculamos la salida del modelo aplicando
        # cada capa de manera secuencial
        for layer in self.layers:
            x = layer(x)
        return x

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#sirve como clase base para construir diferentes capas en una red neuronal.
class Layer():
    def __init__(self):
        self.params = []
        self.grads = []

    def __call__(self, x):
        # por defecto, devolver los inputs
        # cada capa hará algo diferente aquí
        return x

    def backward(self, grad):
        # cada capa, calculará sus gradientes
        # y los devolverá para las capas siguientes
        return grad

    def update(self, params):
        # si hay parámetros, los actualizaremos
        # con lo que nos de el optimizer
        return

In [4]:
class Linear(Layer):
    def __init__(self, d_in, d_out):
        # pesos de la capa inicializa como una matriz aleatoria extraída de una distribución normal con media 0
        #y desviación estándar calculada en función de las dimensiones de entrada y salida.
        self.w = np.random.normal(loc=0.0,
                                  scale=np.sqrt(2/(d_in+d_out)),
                                  size=(d_in, d_out))
        self.b = np.zeros(d_out)

    #realiza el paso de propagación directa del (x) y los pesos self.w, y luego agrega los sesgos self.b.
    def __call__(self, x):
        self.x = x
        self.params = [self.w, self.b]
        # salida del preceptrón
        return np.dot(x, self.w) + self.b
    #desencadena el cálculo de los gradientes
    def backward(self, grad_output):
        # gradientes para la capa siguiente (BACKPROP:minimizar los errores en el proceso de aprendizaje automático de una máquina.)
        grad = np.dot(grad_output, self.w.T)
        self.grad_w = np.dot(self.x.T, grad_output)
        # gradientes para actualizar pesos
        self.grad_b = grad_output.mean(axis=0)*self.x.shape[0]
        self.grads = [self.grad_w, self.grad_b]
        return grad

    def update(self, params):
        self.w, self.b = params

In [5]:
#toma el gradiente de la salida con respecto a la capa subsiguiente
# ( grad_output) ygrad_outputa 0 donde self.xes menor que

class ReLU(Layer):
    def __call__(self, x):
        self.x = x
        return np.maximum(0, x)
#Se crea una máscara booleana grad donde los elementos son True si los elementos
# correspondientes self.x son mayores que cero, y False en caso contrario.
    def backward(self, grad_output):
        grad = self.x > 0
#e calcula el gradiente de retropropagación
        return grad_output*grad
def sigmoid(x):
  return 1 / (1 + np.exp(-x))
# Calcular la exponencial de cada elemento del vector 'x'
def softmax(x):
    return np.exp(x) / np.exp(x).sum(axis=-1,keepdims=True)

class Sigmoid(Layer):
    def __call__(self, x):
        self.x = x
        return sigmoid(x)
#Calcula el gradiente multiplicando grad_output por la derivada de la función sigmoidea aplicada a self.x, que es (sigmoid(self.x) * (1 - sigmoid(self.x))).
    def backward(self, grad_output):
        grad = sigmoid(self.x)*(1 - sigmoid(self.x))
        return grad_output*grad

In [6]:
#se encarga de actualizar los parámetros de una red neuronal utilizando el algoritmo de descenso de gradiente estocástico.
class SGD():
#net se refiere a la red neuronal que se va a entrenar, y lr es la tasa de aprendizaje,
#que determina qué tan rápido se actualizan los pesos de la red durante el entrenamiento.
    def __init__(self, net, lr):
        self.net = net
        self.lr = lr

    def update(self):
        for layer in self.net.layers:
            layer.update([
                params - self.lr*grads
                for params, grads in zip(layer.params, layer.grads)
            ])

In [7]:
#permite calcular los gradientes necesarios para ajustar los pesos de la red neuronal durante el proceso de retropropagación.
class Loss():
    def __init__(self, net):
        self.net = net

    def backward(self):
        # derivada de la loss function con respecto
        # a la salida del MLP
        grad = self.grad_loss()
        # BACKPROPAGATION
        for layer in reversed(self.net.layers):
            grad = layer.backward(grad)

#La clase CrossEntropy hereda de la clase base Loss, implementa la función de pérdida
#de entropía cruzada y su gradiente asociado para un problema de clasificación.
class CrossEntropy(Loss):
    def __call__(self, output, target):
        self.output, self.target = output, target
        logits = output[np.arange(len(output)), target.astype(int)]
        loss = - logits + np.log(np.sum(np.exp(output), axis=-1))
#Finalmente, se toma el promedio de la pérdida calculada
        loss = loss.mean()
        return loss
#se encarga de calcular el gradiente de la función de pérdida.
    def grad_loss(self):
        answers = np.zeros_like(self.output)
        answers[np.arange(len(self.output)), self.target.astype(int)] = 1
#se calcula el gradiente utilizando la fórmula:
        return (- answers + softmax(self.output)) / self.output.shape[0]

In [8]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [10]:
data = pd.read_csv('/content/drive/MyDrive/dataset-tortuga.csv')

In [11]:
print(data)

       Unnamed: 0               NAME   USER_ID  HOURS_DATASCIENCE  \
0              28        Stormy Muto  58283940                7.0   
1              81       Carlos Ferro   1357218               32.0   
2              89  Robby Constantini  63212105               45.0   
3             138       Paul Mckenny  23239851               36.0   
4             143          Jean Webb  72234478               61.0   
...           ...                ...       ...                ...   
19995       20495        Rose Jurado  66754730                0.0   
19996       20496       Johnny Jones   6874888                0.0   
19997       20497    Lawrence Givens  83752787               32.0   
19998       20498    Betty Diclaudio  45806698                0.0   
19999       20499      Connie Harper  67068866               51.0   

       HOURS_BACKEND  HOURS_FRONTEND  NUM_COURSES_BEGINNER_DATASCIENCE  \
0               39.0            29.0                               2.0   
1                0.0   

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 16 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Unnamed: 0                        20000 non-null  int64  
 1   NAME                              20000 non-null  object 
 2   USER_ID                           20000 non-null  int64  
 3   HOURS_DATASCIENCE                 19986 non-null  float64
 4   HOURS_BACKEND                     19947 non-null  float64
 5   HOURS_FRONTEND                    19984 non-null  float64
 6   NUM_COURSES_BEGINNER_DATASCIENCE  19974 non-null  float64
 7   NUM_COURSES_BEGINNER_BACKEND      19982 non-null  float64
 8   NUM_COURSES_BEGINNER_FRONTEND     19961 non-null  float64
 9   NUM_COURSES_ADVANCED_DATASCIENCE  19998 non-null  float64
 10  NUM_COURSES_ADVANCED_BACKEND      19992 non-null  float64
 11  NUM_COURSES_ADVANCED_FRONTEND     19963 non-null  float64
 12  AVG_

In [13]:
#almacenará los nombres de las columnas en un DataFrame data que contengan datos de tipo "object", es decir, columnas que contengan datos categóricos.
columnas_categoricas = data.select_dtypes(include=['object']).columns

In [14]:
#recorre las columnas categóricas en el DataFrame data,
#codifica los valores categóricos utilizando LabelEncoder y reemplaza los valores categóricos con los valores numéricos codificados en el DataFrame.
for columna in columnas_categoricas:
  le = LabelEncoder()
  data[columna] = le.fit_transform(data[columna])

In [15]:
#resultado
print(data)

       Unnamed: 0   NAME   USER_ID  HOURS_DATASCIENCE  HOURS_BACKEND  \
0              28  17600  58283940                7.0           39.0   
1              81   2234   1357218               32.0            0.0   
2              89  15672  63212105               45.0            0.0   
3             138  14783  23239851               36.0           19.0   
4             143   8391  72234478               61.0           78.0   
...           ...    ...       ...                ...            ...   
19995       20495  16295  66754730                0.0           44.0   
19996       20496   9447   6874888                0.0           85.0   
19997       20497  11219  83752787               32.0           50.0   
19998       20498   1590  45806698                0.0           96.0   
19999       20499   3323  67068866               51.0           24.0   

       HOURS_FRONTEND  NUM_COURSES_BEGINNER_DATASCIENCE  \
0                29.0                               2.0   
1                

In [16]:
#se imprime las columnas del dataset
print(data.columns)

Index(['Unnamed: 0', 'NAME', 'USER_ID', 'HOURS_DATASCIENCE', 'HOURS_BACKEND',
       'HOURS_FRONTEND', 'NUM_COURSES_BEGINNER_DATASCIENCE',
       'NUM_COURSES_BEGINNER_BACKEND', 'NUM_COURSES_BEGINNER_FRONTEND',
       'NUM_COURSES_ADVANCED_DATASCIENCE', 'NUM_COURSES_ADVANCED_BACKEND',
       'NUM_COURSES_ADVANCED_FRONTEND', 'AVG_SCORE_DATASCIENCE',
       'AVG_SCORE_BACKEND', 'AVG_SCORE_FRONTEND', 'PROFILE'],
      dtype='object')


In [17]:
#se elimina columnas innecesarias para nuestro entrenamiento
columnas_eiliminar = ['Unnamed: 0', 'USER_ID']
data = data.drop(columnas_eiliminar, axis=1)

In [18]:
#llenar datos vacios
columnas_especificas = ['HOURS_DATASCIENCE', 'HOURS_BACKEND', 'HOURS_FRONTEND', 'NUM_COURSES_BEGINNER_DATASCIENCE', 'NUM_COURSES_BEGINNER_BACKEND', 'NUM_COURSES_BEGINNER_FRONTEND', 'NUM_COURSES_ADVANCED_DATASCIENCE', 'NUM_COURSES_ADVANCED_BACKEND', 'NUM_COURSES_ADVANCED_FRONTEND', 'AVG_SCORE_DATASCIENCE', 'AVG_SCORE_BACKEND', 'AVG_SCORE_FRONTEND']
#se calcula la media de las columnas especificadas utilizando el método mean()
media_columnas = data[columnas_especificas].mean()
#se utiliza el método fillna() para rellenar los valores faltantes en las columnas especificadas.
#Se utiliza media_columnas como argumento para reemplazar los valores faltantes por los valores medios correspondientes.
data[columnas_especificas] = data[columnas_especificas].fillna(media_columnas)

In [19]:
print(data)

        NAME  HOURS_DATASCIENCE  HOURS_BACKEND  HOURS_FRONTEND  \
0      17600                7.0           39.0            29.0   
1       2234               32.0            0.0            44.0   
2      15672               45.0            0.0            59.0   
3      14783               36.0           19.0            28.0   
4       8391               61.0           78.0            38.0   
...      ...                ...            ...             ...   
19995  16295                0.0           44.0            42.0   
19996   9447                0.0           85.0            63.0   
19997  11219               32.0           50.0            22.0   
19998   1590                0.0           96.0            69.0   
19999   3323               51.0           24.0            36.0   

       NUM_COURSES_BEGINNER_DATASCIENCE  NUM_COURSES_BEGINNER_BACKEND  \
0                                   2.0                           4.0   
1                                   2.0                      

In [20]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 14 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   NAME                              20000 non-null  int64  
 1   HOURS_DATASCIENCE                 20000 non-null  float64
 2   HOURS_BACKEND                     20000 non-null  float64
 3   HOURS_FRONTEND                    20000 non-null  float64
 4   NUM_COURSES_BEGINNER_DATASCIENCE  20000 non-null  float64
 5   NUM_COURSES_BEGINNER_BACKEND      20000 non-null  float64
 6   NUM_COURSES_BEGINNER_FRONTEND     20000 non-null  float64
 7   NUM_COURSES_ADVANCED_DATASCIENCE  20000 non-null  float64
 8   NUM_COURSES_ADVANCED_BACKEND      20000 non-null  float64
 9   NUM_COURSES_ADVANCED_FRONTEND     20000 non-null  float64
 10  AVG_SCORE_DATASCIENCE             20000 non-null  float64
 11  AVG_SCORE_BACKEND                 20000 non-null  float64
 12  AVG_

In [21]:
#contendrá un DataFrame con las primeras 13 columnas de data desde 0-12
X = data.iloc[:, :13]
#contendrá una Serie con la columna en la posición 13 de data.
Y = data.iloc[:, 13]

In [22]:
print(X)

        NAME  HOURS_DATASCIENCE  HOURS_BACKEND  HOURS_FRONTEND  \
0      17600                7.0           39.0            29.0   
1       2234               32.0            0.0            44.0   
2      15672               45.0            0.0            59.0   
3      14783               36.0           19.0            28.0   
4       8391               61.0           78.0            38.0   
...      ...                ...            ...             ...   
19995  16295                0.0           44.0            42.0   
19996   9447                0.0           85.0            63.0   
19997  11219               32.0           50.0            22.0   
19998   1590                0.0           96.0            69.0   
19999   3323               51.0           24.0            36.0   

       NUM_COURSES_BEGINNER_DATASCIENCE  NUM_COURSES_BEGINNER_BACKEND  \
0                                   2.0                           4.0   
1                                   2.0                      

In [23]:
#realiza la normalización de los datos en X, almacenando los valores medios y desviaciones estándar
#en X_mean y X_std respectivamente, y el resultado normalizado en X_norm. También devuelve las dimensiones de X y Y.
X_mean, X_std = X.mean(axis=0), X.std(axis=0)
X_norm = (X - X_mean) / X_std

X.shape, Y.shape

((20000, 13), (20000,))

Para ello ya sabemos que tenemos que usar la función de pérdida `CrossEntropy` con función de activación lineal en la última capa.

In [ ]:
#Se definen las dimensiones de entrada (D_in), la cantidad de neuronas en la capa oculta (H), y la dimensión de salida (D_out) de la red neuronal.
D_in, H, D_out = 13, 500, 6
#Se crea la instancia del modelo MLP (mlp) utilizando la clase MLP.Se especifica la arquitectura del modelo, que consiste en capas lineales intercaladas
#con funciones de activación ReLU. La última capa es lineal y tiene una dimensión de salida igual a D_out.
mlp = MLP([
    Linear(D_in, H),
    ReLU(),
    Linear(H, H),
    ReLU(),
    Linear(H, H),
    ReLU(),
    Linear(H, D_out)
])
#Se crea una instancia del optimizador SGD (optimizer) con una tasa de aprendizaje (lr) de 0.02 y se le pasa el modelo MLP (mlp) como argumento.
optimizer = SGD(mlp, lr=0.02)
#Se crea una instancia de la función de pérdida de entropía cruzada (loss) y se le pasa el modelo MLP (mlp) como argumento.
loss = CrossEntropy(mlp)
#Se definen el número de épocas (epochs)
epochs = 500
#tamaño del lote (batch_size).
batch_size = 50
#Se calcula el número de lotes (batches) dividiendo la longitud de X entre el tamaño del lote.
batches = len(X) // batch_size
log_each = 50
#Se crea una lista vacía l para almacenar los valores de pérdida promedio en cada época.
l = []
#Se inicia un bucle for que itera sobre el número de épocas.
for e in range(1,epochs+1):
#se crea una lista vacía _l para almacenar los valores de pérdida en cada lote.
    _l = []
#Se inicia un bucle interno que itera sobre el número de lotes.
    for b in range(batches):
    #se seleccionan los datos de entrada (x) y los objetivos (y) correspondientes al lote actual.
        x = X_norm[b*batch_size:(b+1)*batch_size]
        y = Y[b*batch_size:(b+1)*batch_size]
   #Se obtiene la predicción del modelo MLP (y_pred) para los datos de entrada (x).
        y_pred = mlp(x)
   #Se calcula la pérdida utilizando la función de pérdida de entropía cruzada (loss) pasando la predicción del modelo (y_pred) y los objetivos (y).
   #Se agrega la pérdida del lote actual a la lista _l.
        _l.append(loss(y_pred, y))
   #Se realiza la retropropagación del gradiente de pérdida a través del modelo MLP
   #(loss.backward()) y se actualizan los pesos utilizando el optimizador SGD (optimizer.update()).
        loss.backward()
        optimizer.update()
  #e calcula y se agrega el valor medio de pérdida del lote actual a la lista l.
    l.append(np.mean(_l))
  #Si el número de época actual es divisible por log_each
    if not e % log_each:
  #se imprime el número de época actual y el valor medio de pérdida hasta el momento.
        print(f'Epoch {e}/{epochs}, Loss: {np.mean(l):.4f}')

Epoch 50/500, Loss: 0.1567
Epoch 100/500, Loss: 0.1063


#Nombre: Mikel Alvarez Bejarano
#Carrera: Ing. Sistemas
#Materia: Sis-420
#Examen Segunda instancia
#Fecha: 07/07/2023